In [3]:
import csv
import time
from pylsl import StreamInlet, resolve_byprop
from datetime import datetime

def lsl_to_local_datetime(lsl_timestamp):
    return datetime.fromtimestamp(lsl_timestamp)

# CSVファイルにデータを保存する関数
def save_data_to_csv(timestamp, data, filename):
    fieldnames = ['timestamp', 'AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'gyro_X', 'gyro_Y', 'gyro_Z', 'unknown_1', 'unknown_2']
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # ヘッダーがない場合は、ヘッダーを書き込む
        csvfile.seek(0, 2)
        if csvfile.tell() == 0:
            writer.writeheader()

        row = {'timestamp': lsl_to_local_datetime(timestamp)}
        row.update({fieldnames[i + 1]: data[i] for i in range(len(data))})
        writer.writerow(row)

# EPOC+のストリームを探す
streams = resolve_byprop('type', 'EEG', timeout=5)

if not streams:
    raise RuntimeError("No EEG stream found. Make sure the EmotivPro LSL Connector is running.")

# ストリームを開く
inlet = StreamInlet(streams[0])

print("Connected to Emotiv EPOC+ stream.")

# CSVファイル名を指定
filename = 'epoc_plus_data.csv'

# データを取得し、CSVファイルに記録する
duration = 10  # 収集するデータの期間（秒）
start_time = time.time()

while time.time() - start_time < duration:
    sample, timestamp = inlet.pull_sample()
    save_data_to_csv(timestamp, sample, filename)
    time.sleep(0.1)


Connected to Emotiv EPOC+ stream.


In [2]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

input_filename = 'epoc_plus_data.csv'
output_filename = 'epoc_plus_data_modified.csv'

old_fieldnames = ['timestamp', 'ch_1', 'ch_2', 'ch_3', 'ch_4', 'ch_5', 'ch_6', 'ch_7', 'ch_8', 'ch_9', 'ch_10', 'ch_11', 'ch_12', 'ch_13', 'ch_14', 'ch_15', 'ch_16', 'ch_17', 'ch_18', 'ch_19']
new_fieldnames = ['timestamp', 'AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'gyro_X', 'gyro_Y', 'gyro_Z', 'unknown_1', 'unknown_2']

if not os.path.exists(input_filename):
    print("Input file '{}' does not exist.".format(input_filename))
else:
    with open(input_filename, 'r') as input_csvfile, open(output_filename, 'w', newline='') as output_csvfile:
        reader = csv.DictReader(input_csvfile, fieldnames=old_fieldnames)
        writer = csv.DictWriter(output_csvfile, fieldnames=new_fieldnames)

        # Write the header with the new fieldnames
        writer.writeheader()

        # Skip the header in the input file
        next(reader)

        # Write the remaining rows
        for row in reader:
            writer.writerow({new_fieldnames[i]: row[old_fieldnames[i]] for i in range(len(new_fieldnames))})

    print("Modified CSV file saved as '{}'.".format(output_filename))

# CSVファイルからデータを読み込む関数
def load_data_from_csv(filename):
    data = []
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)

    return data

# データをプロットする関数
def plot_data(data, channels):
    timestamps = [datetime.strptime(row['timestamp'], "%Y-%m-%d %H:%M:%S.%f") for row in data]
    plt.figure(figsize=(15, 8))

    for channel in channels:
        values = [float(row[channel]) for row in data]
        plt.plot(timestamps, values, label=channel)

    plt.xlabel('Timestamp')
    plt.ylabel('Sensor Value')
    plt.title('Brainwave Data')
    plt.legend()
    plt.show()

# 実行
if __name__ == "__main__":
    filename = 'epoc_plus_data_modified.csv'
    data = load_data_from_csv(filename)

    # 選択したチャンネルをプロット
    channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']


Modified CSV file saved as 'epoc_plus_data_modified.csv'.
